In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import matplotlib.pyplot as plt
import pickle
import gzip

In [2]:
from line_profiler import * 

In [5]:

import random

def do_stuff(numbers):
    s = sum(numbers)
    l = [numbers[i]/43 for i in range(len(numbers))]
    m = ['hello'+str(numbers[i]) for i in range(len(numbers))]

numbers = [random.randint(1,100) for i in range(1000)]
do_stuff(numbers)
lp = LineProfiler()
lp.print_stats()

Timer unit: 1e-06 s



In [6]:
from line_profiler import LineProfiler
import random

class Solution:
    def do_stuff(self,numbers):
        s = sum(numbers)
        l = [numbers[i]/43 for i in range(len(numbers))]
        m = ['hello'+str(numbers[i]) for i in range(len(numbers))]

numbers = [random.randint(1,100) for i in range(1000)]

s = Solution()
lp = LineProfiler()
lp_wrapper = lp(s.do_stuff)
lp_wrapper(numbers)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.001129 s
File: <ipython-input-6-6fae3c1c07a7>
Function: do_stuff at line 5

Line #      Hits         Time  Per Hit   % Time  Line Contents
     5                                               def do_stuff(self,numbers):
     6         1         11.0     11.0      1.0          s = sum(numbers)
     7         1        380.0    380.0     33.7          l = [numbers[i]/43 for i in range(len(numbers))]
     8         1        738.0    738.0     65.4          m = ['hello'+str(numbers[i]) for i in range(len(numbers))]



In [7]:
class ClusterBlock(nn.Module):
    def __init__(self, n_input, n_output, v ,T, k_cluster):
        super(ClusterBlock, self).__init__( )
        self.n_input = n_input
        self.n_output = n_output
        self.v = v  # number of node
        self.T = T
        self.k_cluster = k_cluster
        self.linear1 = nn.Linear(n_input , 1)  # Dense
        self.linear2 = nn.Linear(T ,k_cluster ) # clustering
        self.softmax = nn.Softmax(dim =-1)
        # support = 3 , each support has k GAT
        self.attentions1 = [GraphAttentionLayer(T, n_input, n_output) for _ in range(k_cluster)]
        self.attentions2 = [GraphAttentionLayer(T, n_input, n_output) for _ in range(k_cluster)]
        self.attentions3 = [GraphAttentionLayer(T, n_input, n_output) for _ in range(k_cluster)]
        self.dropout = nn.Dropout(p=0.5)
    
    
    def forward(self, x, graph_list):  # x.shape = (b,v,T,f)
        b, *_ = x.size()
        # squeeze f
        out = self.linear1(x)    #  out.shape = (b , v ,T,1)
        out = out.view(-1, self.v, self.T)  #  out.shape = (b , v ,T)

        # clustering
        soft_cluster = self.softmax(self.linear2(out))   # out.shape = (b,v,k)
        
        # soft to hard        
#         m = torch.transpose(torch.max(soft_cluster, -1)[0].repeat(2,1),0,1)
        max_mat = torch.max(soft_cluster, -1)[0].unsqueeze(-1).repeat(1, 1, self.k_cluster)
        one_mat = torch.ones(b, self.v, self.k_cluster)
        zero_mat = torch.zeros(b, self.v, self.k_cluster)
        hard_cluster = torch.where(soft_cluster-max_mat>=0, one_mat, zero_mat)
        
        #graph attention
        gout = 0
        for graph in graph_list:   # graph_list : (3,3 ,n , n )
            graph1, graph2, graph3 = graph[0], graph[1], graph[2]
#             cluster_mask_out = torch.zeros(3, b, self.v, self.T*self.output)
            for i in range(self.k_cluster):
                out1 = torch.mul(self.attentions1[i](x, graph1), hard_cluster[:, :, i].unsqueeze(-1))
                out2 = torch.mul(self.attentions2[i](x, graph1), hard_cluster[:, :, i].unsqueeze(-1))
                out3 = torch.mul(self.attentions2[i](x, graph1), hard_cluster[:, :, i].unsqueeze(-1))
                out = F.relu((out1 + out2  + out3)/3 )
            gout += out
#                 cluster_mask_out[0] += out1
#                 cluster_mask_out[1] += out2
#                 cluster_mask_out[2] += out3
#             gout.append(torch.sum(cluster_mask_out, dim=0))

        return gout/3 , hard_cluster 

In [8]:
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, T , in_features, out_features, concat=False):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = nn.Dropout(p=0.5)
        self.in_features = in_features
        self.out_features = out_features
        self.concat = concat
        self.W = nn.Parameter(torch.zeros(size=(T*in_features, T*out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*T*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU()
        

    def forward(self, input, adj): # input.shape = ( b,v, t ,f1)
        B , N , T = input.size()[0], input.size()[1], input.size()[2]     
        
        # h = xw , (b,N,t*f1)  * ( t*f1 ,  t*f2) -> (b ,N , t*f2)
        input = input.view(B*N, -1)
        h = torch.matmul(input, self.W).view(B , N , -1)
        
        # h.repeat(1,1,N) : (b, N, N*t*f2)  
        # h.repeat(1,N,1) : (b ,N*N , t*f2)  
        # output : (b , N , N, 2*t*f2)
        a_input = torch.cat([h.repeat(1, 1, N).view(B, N * N, -1), h.repeat(1, N, 1)], dim=1).view(B, N, N, 2*T*self.out_features)
        # e = a_input* a ,  (b,N,N,2*t*f2) * ( 2*t*f2 ,1) -> ( b ,N ,N )
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(-1))
        zero_vec = -9e15*torch.ones_like(e)
        # adj : (n, n) -> ( b , n ,n )
        b_adj = adj.unsqueeze(0).repeat(B, 1, 1)
        
        attention = torch.where(b_adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=-1)
#         attention = self.dropout(attention, self.dropout, training=self.training)
        
        # (b,n,n) * (b,n ,t*f2)  -> (b, n , t*f2)
        h_prime = torch.bmm(attention, h)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    
    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


In [9]:
test_data = torch.rand(10 , 20 , 48 ,1)  # b,v,t, f
test_label = torch.rand(10, 20 , 48 ,1)
graph = torch.rand(20,20)
graph_list = torch.rand(3,3,20,20)

In [10]:
T = 48
in_features = 1 
out_features = 1

In [11]:
model_layer = GraphAttentionLayer(T , in_features, out_features)
model_layer(test_data,graph)

tensor([[[-0.0434, -1.1207,  0.2787,  ..., -0.3739, -0.7780,  0.9089],
         [ 0.1045, -1.2020,  0.2528,  ..., -0.3410, -0.7588,  0.8207],
         [ 0.0497, -1.1719,  0.2624,  ..., -0.3532, -0.7659,  0.8534],
         ...,
         [-0.0233, -1.1288,  0.2170,  ..., -0.4523, -0.6655,  0.7165],
         [-0.0233, -1.1288,  0.2170,  ..., -0.4523, -0.6655,  0.7165],
         [-0.0233, -1.1288,  0.2170,  ..., -0.4523, -0.6655,  0.7165]],

        [[-0.0661, -1.1882,  0.1886,  ..., -0.2305, -0.8488,  0.8532],
         [-0.0656, -1.1889,  0.1867,  ..., -0.2228, -0.8516,  0.8696],
         [-0.0672, -1.1864,  0.1933,  ..., -0.2486, -0.8420,  0.8148],
         ...,
         [ 0.0019, -1.0853,  0.2172,  ..., -0.2019, -0.8863,  0.9481],
         [ 0.0019, -1.0853,  0.2172,  ..., -0.2019, -0.8863,  0.9481],
         [ 0.0019, -1.0853,  0.2172,  ..., -0.2019, -0.8863,  0.9481]],

        [[-0.0337, -1.2022,  0.4296,  ..., -0.2032, -0.9635,  1.0024],
         [-0.0284, -1.2067,  0.4424,  ..., -0

In [12]:
from line_profiler import LineProfiler

model_layer = GraphAttentionLayer(T , in_features, out_features)
# model_layer(test_data,graph)


lp = LineProfiler()
lp_wrapper = lp(model_layer.forward)
lp_wrapper(test_data,graph)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.009483 s
File: <ipython-input-8-20f176e5c274>
Function: forward at line 20

Line #      Hits         Time  Per Hit   % Time  Line Contents
    20                                               def forward(self, input, adj): # input.shape = ( b,v, t ,f1)
    21         1          8.0      8.0      0.1          B , N , T = input.size()[0], input.size()[1], input.size()[2]     
    22                                                   
    23                                                   # h = xw , (b,N,t*f1)  * ( t*f1 ,  t*f2) -> (b ,N , t*f2)
    24         1         17.0     17.0      0.2          input = input.view(B*N, -1)
    25         1       2774.0   2774.0     29.3          h = torch.matmul(input, self.W).view(B , N , -1)
    26                                                   
    27                                                   # h.repeat(1,1,N) : (b, N, N*t*f2)  
    28                                                   # h.repeat(1,N,

In [125]:
T  = 48
n_input = 1 
n_output = 1 
v = 20 
k_cluster = 3

In [126]:
model = ClusterBlock(n_input, n_output, v ,T, k_cluster)

In [127]:
lp = LineProfiler()
lp_wrapper = lp(model.forward)
lp_wrapper(test_data,graph_list)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.003513 s
File: <ipython-input-111-73d9242b4732>
Function: forward at line 19

Line #      Hits         Time  Per Hit   % Time  Line Contents



In [107]:
model(test_data, graph_list)[0]

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0057, 0.0000, 0.7051,  ..., 0.0000, 0.3279, 0.1443],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7105,  ..., 0.0000, 0.3549, 0.1563],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.2179, 0.0000, 0.6480,  ..., 0.0000, 0.2186, 0.1773],
         ...,
         [0.2328, 0.0000, 0.6930,  ..., 0.0000, 0.2687, 0.1587],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.1547, 0.0000, 0.8107,  ..., 0.0000, 0.3011, 0.0470],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.